In [1]:
import numpy as np
import pandas as pd
import torch
import tensorly as tl
# library for TTD   (Tensorly  (search))

from tensorly.decomposition import parafac
from tensorly.decomposition import tucker
from tensorly.decomposition import tensor_train
from tensorly.decomposition import tensor_train
# different forms of tensor
from sklearn.preprocessing import MinMaxScaler
from tensorly import tt_to_tensor
from tensorly.decomposition import matrix_product_state
# tensor operations


#from math import ceil
#from google.colab import drive
#drive.mount('/content/gdrive')
import torch.nn.init as init
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms, utils
from torch.utils.data import Dataset, DataLoader
import time
import os
import copy
import torch.nn.functional as Fun
print("PyTorch Version:", torch.__version__)
print("Torchvision Version:", torchvision.__version__)
print("GPU is available?", torch.cuda.is_available())

PyTorch Version: 1.13.1
Torchvision Version: 0.14.1
GPU is available? True


In [2]:
dtype = torch.float
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
features = list()
with open("/home/c/cl237/TenBCD/UCI HAR/data/features.txt") as f:
    features = [line.split()[1] for line in f.readlines()]
print('# of Features: {}'.format(len(features)))

# of Features: 561


## get train data

In [4]:
pd_X_train = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/X_train.txt', delim_whitespace=True, header=None)
pd_X_train.columns = features
pd_y_train = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/y_train.txt', delim_whitespace=True, names=['Activity'])

N = len(pd_X_train)

K = 6

x = pd_X_train.values
y = pd_y_train.values

X_train=torch.tensor(x,dtype = torch.float32,device=device)
y_train=torch.tensor(y,dtype = int) #dtype=torch.int,device=device)

y_train = torch.flatten(y_train)

X_train = torch.t(X_train)

y_one_hot = torch.zeros(N, K).scatter_(1, torch.reshape(y_train-1, (N, 1)), 1)
y_one_hot = torch.t(y_one_hot).to(device=device)
y_train = y_train.to(device=device)

## get test data

In [5]:
pd_X_test = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/X_test.txt', delim_whitespace=True, header=None)
pd_X_test.columns = features
pd_y_test = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/y_test.txt', delim_whitespace=True, names=['Activity'])

N_test = len(pd_X_test)

x = pd_X_test.values
y = pd_y_test.values

X_test=torch.tensor(x,dtype = torch.float32, device=device)
y_test=torch.tensor(y,dtype = int)

y_test = torch.flatten(y_test)

X_test = torch.t(X_test)

y_test_one_hot = torch.zeros(N_test, K).scatter_(1, torch.reshape(y_test-1, (N_test, 1)), 1)
y_test_one_hot = torch.t(y_test_one_hot).to(device=device)
y_test = y_test.to(device=device)

In [6]:
def updateV(U1,U2,W,b,rho,gamma): 
    _, d = W.size()
    I = torch.eye(d, device=device) #Returns a 2-D tensor with ones on the diagonal and zeros elsewhere.
    U1 = nn.ReLU()(U1)
    _, col_U2 = U2.size()
    Vstar = torch.mm(torch.inverse(rho*(torch.mm(torch.t(W),W))+gamma*I), rho*torch.mm(torch.t(W),U2-b.repeat(1,col_U2))+gamma*U1)
    # check how to get V* W*
    # torch.mm*=(input,mat2) performs a matrix multiplication of the matrices input and mat2.
    #   if input is a (n*m) tensor,mat2 is a (m*p) tensor, output will be (n*p) tensor
    return Vstar

def updateWb_org(U, V, W, b, alpha, rho): 
    d,N = V.size()
    I = torch.eye(d, device=device)
    _, col_U = U.size()
    Wstar = torch.mm(alpha*W+rho*torch.mm(U-b.repeat(1,col_U),torch.t(V)),torch.inverse(alpha*I+rho*(torch.mm(V,torch.t(V)))))
    bstar = 0*(alpha*b+rho*torch.sum(U-torch.mm(Wstar,V), dim=1).reshape(b.size()))/(rho*N+alpha)
    return Wstar, bstar

def updateWb(U, V, W, b, W_tensor_rec, alpha, rho,tau): 
    W_tensor_rec = torch.as_tensor(W_tensor_rec,device=device).float()
    W_tensor2matrix = W_tensor_rec.reshape(W.shape)
    d,N = V.size()
    I = torch.eye(d, device=device)
    _, col_U = U.size()
    Wstar = torch.mm(alpha*W+tau*W_tensor2matrix+rho*torch.mm(U-b.repeat(1,col_U),torch.t(V)),torch.inverse((alpha+tau)*I+rho*(torch.mm(V,torch.t(V)))))
    bstar = 0*(alpha*b+rho*torch.sum(U-torch.mm(Wstar,V), dim=1).reshape(b.size()))/(rho*N+alpha)
    return Wstar, bstar

# U update  E(1) in global convergent
def relu_prox(a, b, gamma, d, N):
    val = torch.empty(d,N, device=device)
    x = (a+gamma*b)/(1+gamma)
    y = torch.min(b,torch.zeros(d,N, device=device))

    val = torch.where(a+gamma*b < 0, y, torch.zeros(d,N, device=device))
    val = torch.where(((a+gamma*b >= 0) & (b >=0)) | ((a*(gamma-np.sqrt(gamma*(gamma+1))) <= gamma*b) & (b < 0)), x, val)
    val = torch.where((-a <= gamma*b) & (gamma*b <= a*(gamma-np.sqrt(gamma*(gamma+1)))), b, val)
    return val

In [8]:
########### parameter setup
#df = pd.DataFrame()
#df.to_csv('C:/Users/Mark/Desktop/rank40_100times.csv')
niter = 500
rank = 80
tau = 5
alpha = 1
gamma = 0.5
rho = 0.5

#alpha1 = alpha2 = alpha3 = alpha4 = alpha5 = alpha6 = alpha7 = alpha8 = alpha9 = alpha10 = alpha
#for rank in (35,): #(25,30,35)(100,180,220,260,300,340,380)(20,60,100,140,180,220,260,300,340,380)
#    for tau in (400,500): #(300,400,500)(10,50,100,200,300)(10,50,100,200,300)
#        for gamma in (0.5,0.8,2): #(0.5,0.8,2)(0.5,0.8)(0.5,1,1.5,2,3)
            #gamma1 = gamma2 = gamma3 = gamma4 = gamma5 = gamma
#            for rho in (0.5,0.8,2): #(0.5,0.8)(1,2)
                #rho1 = rho2 = rho3 = rho4 = rho5= rho
#                for alpha in (0.5,1,1.5,2):
#                    print ("rank=",rank, "tau=",tau, "gamma=",gamma, "rho=",rho, "alpha",alpha)
                    #print('Compression Ratio', ((1024*28*28+10*1024+(8*(rank)+32*np.square(rank))*2)/(1024*28*28+10*1024+1024*1024*2)), (8*(rank)+32*np.square(rank))*2/(1024*1024*2))
        
print ("rank=",rank, "tau=",tau, "gamma=",gamma, "rho=",rho, "alpha",alpha)

loss1 = np.empty(niter)
loss2 = np.empty(niter)
accuracy_train = np.empty(niter)
accuracy_test = np.empty(niter)
time1 = np.empty(niter)

results = torch.zeros(1, 5, niter)



for Out_iter in range(1):
    rank_initial = 400
    seed = 10 + 10*Out_iter
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.manual_seed(seed)

    d0 = 561 #561 =3*11*17
    d1 =  1024    # 2^10
    d2 =  1024  
    d3 = 1024
    d4 = 512      # 2^9
    d5 = 6 

    std_W1 = torch.sqrt(torch.tensor(1. / d0))
    W1 =0.2* torch.empty(d1, d0, device=device).normal_(0, std_W1)
    W1_torch_tensor = W1.reshape((6,22,34,2,2,2,2,2,2,2))
    W1_tl_tensor = tl.tensor(W1_torch_tensor.cpu().numpy())
    factors1 = tensor_train(W1_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W1_tl_tensor_rec = tt_to_tensor(factors1)
    b1 = 0*torch.ones(d1, 1, device=device) # 0 is stable


    std_W2 = torch.sqrt(torch.tensor(1. / d1))
    W2 =0.2* torch.empty(d2, d1, device=device).normal_(0, std_W2)
    W2_torch_tensor = W2.reshape((4,4,4,4,4,4,4,4,4,4)) # 8 number of 4s, 2 number of 8s
    W2_tl_tensor = tl.tensor(W2_torch_tensor.cpu().numpy())
    factors2 = tensor_train(W2_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    # we have 9 rank_initial (r1,...,r9) here plus 2 of 1s (r0 & r10).
    # factors: set of tensor cores
    # http://tensorly.org/stable/user_guide/tensor_decomposition.html
    # http://tensorly.org/stable/modules/generated/tensorly.decomposition.tensor_train.html#tensorly.decomposition.tensor_train
    W2_tl_tensor_rec = tt_to_tensor(factors2)
    # tt_to_tensor(factors):Re-assembles ‘factors’, which represent a tensor in TT format into the corresponding full tensor
    #      facros: list of 3d-arrays tt-cores           output_tensor: ndarray      
    b2 = 0*torch.ones(d2, 1, device=device)


    std_W3 = torch.sqrt(torch.tensor(1. / d2))
    W3 =0.2* torch.empty(d3, d2, device=device).normal_(0, std_W3)
    W3_torch_tensor = W3.reshape((4,4,4,4,4,4,4,4,4,4)) # 8 number of 4s, 2 number of 8s
    W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())
    factors3 = tensor_train(W3_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W3_tl_tensor_rec = tt_to_tensor(factors3)
    b3 = 0*torch.ones(d3, 1, device=device)

    std_W4 = torch.sqrt(torch.tensor(1. / d3))
    W4 =0.2* torch.empty(d4, d3, device=device).normal_(0, std_W4)
    W4_torch_tensor = W4.reshape((4,4,4,4,4,4,4,4,4,2)) # 8 number of 4s, 2 number of 8s
    W4_tl_tensor = tl.tensor(W4_torch_tensor.cpu().numpy())
    factors4 = tensor_train(W4_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W4_tl_tensor_rec = tt_to_tensor(factors4)
    b4 = 0*torch.ones(d4, 1, device=device)


    std_W5 = torch.sqrt(torch.tensor(1. / d4))
    W5 =0.2* torch.empty(d5, d4, device=device).normal_(0, std_W5)
    b5 = 0*torch.ones(d5, 1, device=device)

    # W4 = 0.01*torch.randn(d4, d3, device=device)
    # W4_torch_tensor = W4.reshape((50,30,K))
    # W4_tl_tensor = tl.tensor(W4_torch_tensor.cpu().numpy())
    # core4, tucker_factors4 = tucker(W4_tl_tensor, rank=(40,28,K), init='svd', tol=10e-5, random_state=12345)
    # W4_tl_tensor_rec = tl.tucker_to_tensor((core4, tucker_factors4))
    # b4 = 0.1*torch.ones(d4, 1, device=device)

    U1 = torch.addmm(b1.repeat(1, N), W1, X_train)
    V1 = nn.ReLU()(U1)
    U2 = torch.addmm(b2.repeat(1, N), W2, V1)
    V2 = nn.ReLU()(U2)
    U3 = torch.addmm(b3.repeat(1, N), W3, V2)
    V3 = nn.ReLU()(U3)
    U4 = torch.addmm(b4.repeat(1, N), W4, V3)
    V4 = nn.ReLU()(U4)
    U5 = torch.addmm(b5.repeat(1, N), W5, V4)
    V5 = U5 
    # U4 = torch.addmm(b4.repeat(1, N), W4, V3)
    # V4 = U4



    # Iterations
    print('Train on', N, 'samples, validate on', N_test, 'samples')
    for k in range(niter):
        start = time.time()

 # update for last layer
        # update V4
        V5 = (y_one_hot + gamma*U5 + alpha*V5)/(1 + gamma + alpha)

        # update U4 
        U5 = (gamma*V5 + rho*(torch.mm(W5,V4) + b5.repeat(1,N)))/(gamma + rho)

        # update W4 and b4
        W5, b5 = updateWb_org(U5,V4,W5,b5,alpha,rho)
        # W3_torch_tensor = W3.reshape((d3,4,4,4,4,4,4))
        # W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())
        # factors3 = tensor_train(W3_tl_tensor, (1,rank,rank,rank,rank,rank,rank,1))
        # W3_tl_tensor_rec = tt_to_tensor(factors3)



  # update for 4th layer
        # update V3
        V4 = updateV(U4,U5,W5,b5,rho,gamma)

        # update U3
        U4 = relu_prox(V4,(rho*torch.addmm(b4.repeat(1,N), W4, V3) + alpha*U4)/(rho + alpha),(rho + alpha)/gamma,d4,N)

        # update W3 and b3
        W4, b4 = updateWb(U4,V3,W4,b4,W4_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)   
        W4_torch_tensor = W4.reshape((4,4,4,4,4,4,4,4,4,2))
        W4_tl_tensor = tl.tensor(W4_torch_tensor.cpu().numpy())  # transfer tensorly package
        factors4 = tensor_train(W4_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        #set of tensor cores
        W4_tl_tensor_rec = tt_to_tensor(factors4)

  # update for 3nd layer
        # update V3
        V3 = updateV(U3,U4,W4,b4,rho,gamma)

        # update U3
        U3 = relu_prox(V3,(rho*torch.addmm(b3.repeat(1,N), W3, V2) + alpha*U3)/(rho + alpha),(rho + alpha)/gamma,d3,N)

        # update W3 and b3
        W3, b3 = updateWb(U3,V2,W3,b3,W3_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)   
        W3_torch_tensor = W3.reshape((4,4,4,4,4,4,4,4,4,4))
        W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())  # transfer tensorly package
        factors3 = tensor_train(W3_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        #set of tensor cores
        W3_tl_tensor_rec = tt_to_tensor(factors3)

 # update for 2nd layer
        # update V2
        V2 = updateV(U2,U3,W3,b3,rho,gamma)

        # update U2
        U2 = relu_prox(V2,(rho*torch.addmm(b2.repeat(1,N), W2, V1) + alpha*U2)/(rho + alpha),(rho + alpha)/gamma,d2,N)

        # update W2 and b2
        W2, b2 = updateWb(U2,V1,W2,b2,W2_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)   
        W2_torch_tensor = W2.reshape((4,4,4,4,4,4,4,4,4,4))
        W2_tl_tensor = tl.tensor(W2_torch_tensor.cpu().numpy()) 
        factors2 = tensor_train(W2_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        W2_tl_tensor_rec = tt_to_tensor(factors2)


# update for 1st layer
        # update V1
        V1 = updateV(U1,U2,W2,b2,rho,gamma)

        # update U1
        U1 = relu_prox(V1,(rho*torch.addmm(b1.repeat(1,N), W1, X_train) + alpha*U1)/(rho + alpha),(rho + alpha)/gamma,d1,N)

        # update W1 and b1
        W1, b1 = updateWb(U1,X_train,W1,b1,W1_tl_tensor_rec, alpha,rho,tau)

        # G update
        W1_torch_tensor = W1.reshape((6,22,34,2,2,2,2,2,2,2))
        W1_tl_tensor = tl.tensor(W1_torch_tensor.cpu().numpy())
        factors1 = tensor_train(W1_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        W1_tl_tensor_rec = tt_to_tensor(factors1)




 # prediction for trainning data
        a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), X_train))
        #a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), W1, x_train))
        a2_train = nn.ReLU()(torch.addmm(b2.repeat(1, N), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_train))
        a3_train = nn.ReLU()(torch.addmm(b3.repeat(1, N), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_train))
        a4_train = nn.ReLU()(torch.addmm(b4.repeat(1, N), torch.as_tensor(W4_tl_tensor_rec,device=device).reshape((d4, d3)).float(), a3_train))
        #print(torch.addmm(b4.repeat(1, N), W4, a3_train))
        pred = torch.argmax(torch.addmm(b5.repeat(1, N), W5, a4_train), dim=0)
        # check argmax and addmm, dim=0

 #Prediction for test data
        a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), X_test))
        #a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), W1, x_test))
        a2_test = nn.ReLU()(torch.addmm(b2.repeat(1, N_test), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_test))
        a3_test = nn.ReLU()(torch.addmm(b3.repeat(1, N_test), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_test))
        a4_test = nn.ReLU()(torch.addmm(b4.repeat(1, N_test), torch.as_tensor(W4_tl_tensor_rec,device=device).reshape((d4, d3)).float(), a3_test))
        pred_test = torch.argmax(torch.addmm(b5.repeat(1, N_test), W5, a4_test), dim=0)

        # a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), x_train))
        # a2_train = nn.ReLU()(torch.addmm(b2.repeat(1, N), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_train))
        # # a3_train = nn.ReLU()(torch.addmm(b3.repeat(1, N), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)), a2_train))
        # #print(torch.addmm(b4.repeat(1, N), W4, a3_train))
        # pred = torch.argmax(torch.addmm(b3.repeat(1, N), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_train), dim=0)

        # a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), x_test))
        # a2_test = nn.ReLU()(torch.addmm(b2.repeat(1, N_test), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_test))
        # # a3_test = nn.ReLU()(torch.addmm(b3.repeat(1, N_test), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)), a2_test))
        # pred_test = torch.argmax(torch.addmm(b3.repeat(1, N_test), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_test), dim=0)

        # a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), W1, x_train))
        # a2_train = nn.ReLU()(torch.addmm(b2.repeat(1, N), W2, a1_train))

        # #print(torch.addmm(b4.repeat(1, N), W4, a3_train))
        # pred = torch.argmax(torch.addmm(b3.repeat(1, N), W3, a2_train), dim=0)

        # a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), W1, x_test))
        # a2_test = nn.ReLU()(torch.addmm(b2.repeat(1, N_test), W2, a1_test))
        # pred_test = torch.argmax(torch.addmm(b3.repeat(1, N_test), W3, a2_test), dim=0)




   #emperical loss
        loss1[k] = gamma/2*torch.pow(torch.dist(V5,y_one_hot,2),2).cpu().numpy()
        # torch.pow:Takes the power of each element in input with exponent and returns a tensor with the result.
        # torch.disk: Returns the p-norm of (input - other)
        # Eq (5) in paper
        loss2[k] = loss1[k] + rho/2*torch.pow(torch.dist(torch.addmm(b1.repeat(1,N), W1, X_train),U1,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b2.repeat(1,N), W2, V1),U2,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b3.repeat(1,N), W3, V2),U3,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b4.repeat(1,N), W4, V3),U4,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b5.repeat(1,N), W5, V4),U5,2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V1,nn.ReLU()(U1),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V2,nn.ReLU()(U2),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V3,nn.ReLU()(U3),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V4,nn.ReLU()(U4),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V5,U5,2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W1.reshape((6,22,34,2,2,2,2,2,2,2)),torch.as_tensor(W1_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W2.reshape((4,4,4,4,4,4,4,4,4,4)),torch.as_tensor(W2_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W3.reshape((4,4,4,4,4,4,4,4,4,4)),torch.as_tensor(W3_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W4.reshape((4,4,4,4,4,4,4,4,4,2)),torch.as_tensor(W4_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \

        # +tau/2*torch.pow(torch.dist(W3.reshape((d3,4,4,4,4,4,4)),torch.as_tensor(W3_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() 
        # +tau/2*torch.pow(torch.dist(W4.reshape((50,30,K)),torch.as_tensor(W4_tl_tensor_rec,device=device),2),2).cpu().numpy() 

        # compute training accuracy
        correct_train = pred == y_train-1
        accuracy_train[k] = np.mean(correct_train.cpu().numpy())

        # compute validation accuracy
        correct_test = pred_test == y_test-1
        accuracy_test[k] = np.mean(correct_test.cpu().numpy())

        # compute training time
        stop = time.time()
        duration = stop - start
        time1[k] = duration

        # print results
        print('Repeatition', Out_iter + 1, 'Epoch', k + 1, '/', niter, '\n', 
              '-', 'time:', time1[k], '-', 'sq_loss:', loss1[k], '-', 'tot_loss:', loss2[k], 
              '-', 'acc:', accuracy_train[k], '-', 'val_acc:', accuracy_test[k])

##############
############## compute CR
    factors1_shape=[f.shape for f in factors1]
    Sum_of_variables_factors1=sum(list(x*y*z for x,y,z in factors1_shape))
    factors2_shape=[f.shape for f in factors2]
    Sum_of_variables_factors2=sum(list(x*y*z for x,y,z in factors2_shape))
    factors3_shape=[f.shape for f in factors3]
    Sum_of_variables_factors3=sum(list(x*y*z for x,y,z in factors3_shape))
    factors4_shape=[f.shape for f in factors4]
    Sum_of_variables_factors4=sum(list(x*y*z for x,y,z in factors4_shape))

    total_variabels=Sum_of_variables_factors1+Sum_of_variables_factors2+Sum_of_variables_factors3+Sum_of_variables_factors4

    CR_1=((total_variabels)+(d4*d5))/(d0*d1+d1*d2+d2*d3+d3*d4+d4*d5)
    CR_2=(total_variabels)/(d0*d1+d1*d2+d2*d3+d3*d4)
    print("CR_1 = "+str(CR_1)+"   "+"CR_2 = " +str(CR_2))


    results[Out_iter,0,:] = torch.tensor(loss1)
    results[Out_iter,1,:] = torch.tensor(loss2)
    results[Out_iter,2,:] = torch.tensor(accuracy_train)
    results[Out_iter,3,:] = torch.tensor(accuracy_test)
    results[Out_iter,4,:] = torch.tensor(time1)
    CR=(CR_1,CR_2)

#this postion to add new row into existing table
    #df=pd.read_csv('C:/Users/Mark/Desktop/rank40_100times.csv')
    #new_row = {'rank':rank, 'CR_1':CR_1, 'CR_2':CR_2, 'tau':tau, 'gamma':gamma, 'rho':rho, 'alpha':alpha, \
    #           'loss1':loss1[niter-1], 'loss2':loss2[niter-1], 'accuracy_train':accuracy_train[niter-1], \
    #           'accuracy_test':accuracy_test[niter-1], 'time':time1[niter-1], 'seed':seed} 
    #df=df.append(new_row,ignore_index=True)
    #df.to_csv('C:/Users/Mark/Desktop/rank40_100times.csv',index=False)


filename= "LecunNormal_" + "niter_"+ str(niter) + "rank_" + str(rank) + "tau_" + str(tau) + "gamma_" + str(gamma) + \
"rho_" + str(rho) + "alpha_" + str(alpha) + ".mat"
from scipy.io import savemat
%cd /home/c/cl237/TenBCD/UCI HAR/4 hidden layers/Different Initialization/
savemat (filename, {'results': torch.Tensor.numpy(results), 'Compression Ratio':CR})
#this position to save table into matlab
#df.to_csv('C:/Users/Mark/Desktop/result_compress_123_layer_highrank.csv')
##Changing Folder
 #%cd '/content/gdrive/MyDrive/Colab/Tensor-BCD-for-DNN/Experiments/DataSaved'
 #savemat("ThreeLayer_rank160.mat", {'results': torch.Tensor.numpy(results)})    

rank= 80 tau= 5 gamma= 0.5 rho= 0.5 alpha 1
Train on 7352 samples, validate on 2947 samples
Repeatition 1 Epoch 1 / 500 
 - time: 1.278249740600586 - sq_loss: 661.6724853515625 - tot_loss: 1028.818889358321 - acc: 0.1868879216539717 - val_acc: 0.18052256532066507
Repeatition 1 Epoch 2 / 500 
 - time: 1.2605726718902588 - sq_loss: 294.0766296386719 - tot_loss: 485.82475444767624 - acc: 0.1868879216539717 - val_acc: 0.18052256532066507
Repeatition 1 Epoch 3 / 500 
 - time: 1.2398607730865479 - sq_loss: 161.25238037109375 - tot_loss: 268.10302961058915 - acc: 0.1868879216539717 - val_acc: 0.18052256532066507
Repeatition 1 Epoch 4 / 500 
 - time: 1.2554261684417725 - sq_loss: 87.3482666015625 - tot_loss: 152.4793573366478 - acc: 0.1868879216539717 - val_acc: 0.18052256532066507
Repeatition 1 Epoch 5 / 500 
 - time: 1.2432985305786133 - sq_loss: 46.95746994018555 - tot_loss: 89.90317907370627 - acc: 0.1868879216539717 - val_acc: 0.18052256532066507
Repeatition 1 Epoch 6 / 500 
 - time: 1.26

Repeatition 1 Epoch 48 / 500 
 - time: 1.2240910530090332 - sq_loss: 0.0005614866968244314 - tot_loss: 1.1465305273450213 - acc: 0.888873775843308 - val_acc: 0.8713946386155412
Repeatition 1 Epoch 49 / 500 
 - time: 1.2276592254638672 - sq_loss: 0.0005240177270025015 - tot_loss: 1.0967890536285267 - acc: 0.89050598476605 - val_acc: 0.8730912792670512
Repeatition 1 Epoch 50 / 500 
 - time: 1.2236816883087158 - sq_loss: 0.0004895291640423238 - tot_loss: 1.0425468868997996 - acc: 0.8936343852013058 - val_acc: 0.8761452324397693
Repeatition 1 Epoch 51 / 500 
 - time: 1.2302687168121338 - sq_loss: 0.000457661779364571 - tot_loss: 0.9955724976862257 - acc: 0.8954026115342764 - val_acc: 0.8764845605700713
Repeatition 1 Epoch 52 / 500 
 - time: 1.2179327011108398 - sq_loss: 0.00042797272908501327 - tot_loss: 0.9610044298569846 - acc: 0.8975788900979326 - val_acc: 0.8802171700033933
Repeatition 1 Epoch 53 / 500 
 - time: 1.2267773151397705 - sq_loss: 0.0004005588998552412 - tot_loss: 0.91685582

Repeatition 1 Epoch 94 / 500 
 - time: 1.2271745204925537 - sq_loss: 4.629102113540284e-05 - tot_loss: 0.2730415819751215 - acc: 0.948993471164309 - val_acc: 0.9389209365456397
Repeatition 1 Epoch 95 / 500 
 - time: 1.2301928997039795 - sq_loss: 4.452470602700487e-05 - tot_loss: 0.2626119409284229 - acc: 0.9494015233949945 - val_acc: 0.9395995928062436
Repeatition 1 Epoch 96 / 500 
 - time: 1.2353408336639404 - sq_loss: 4.287187766749412e-05 - tot_loss: 0.25895429881745713 - acc: 0.9504896626768227 - val_acc: 0.9402782490668476
Repeatition 1 Epoch 97 / 500 
 - time: 1.2346460819244385 - sq_loss: 4.128071304876357e-05 - tot_loss: 0.25175482797214954 - acc: 0.9514417845484222 - val_acc: 0.9409569053274517
Repeatition 1 Epoch 98 / 500 
 - time: 1.228027582168579 - sq_loss: 3.98117299482692e-05 - tot_loss: 0.2491811586140784 - acc: 0.9521218715995647 - val_acc: 0.9409569053274517
Repeatition 1 Epoch 99 / 500 
 - time: 1.2401933670043945 - sq_loss: 3.8394042348954827e-05 - tot_loss: 0.24218

Repeatition 1 Epoch 140 / 500 
 - time: 1.262101411819458 - sq_loss: 1.4371717952599283e-05 - tot_loss: 0.14259193111530521 - acc: 0.9710282916213275 - val_acc: 0.9548693586698337
Repeatition 1 Epoch 141 / 500 
 - time: 1.2575187683105469 - sq_loss: 1.4184650353854522e-05 - tot_loss: 0.14646698387869606 - acc: 0.9710282916213275 - val_acc: 0.9552086868001357
Repeatition 1 Epoch 142 / 500 
 - time: 1.222520351409912 - sq_loss: 1.3995626432006247e-05 - tot_loss: 0.13723831758241545 - acc: 0.971436343852013 - val_acc: 0.9552086868001357
Repeatition 1 Epoch 143 / 500 
 - time: 1.2586424350738525 - sq_loss: 1.3808436960971449e-05 - tot_loss: 0.14044637250151482 - acc: 0.9715723612622416 - val_acc: 0.9548693586698337
Repeatition 1 Epoch 144 / 500 
 - time: 1.257127285003662 - sq_loss: 1.3625073734147009e-05 - tot_loss: 0.14862841860588105 - acc: 0.9717083786724701 - val_acc: 0.9548693586698337
Repeatition 1 Epoch 145 / 500 
 - time: 1.2379124164581299 - sq_loss: 1.3453839528665412e-05 - tot_

Repeatition 1 Epoch 186 / 500 
 - time: 1.2372446060180664 - sq_loss: 9.512520591670182e-06 - tot_loss: 0.11153463821709408 - acc: 0.9789173014145811 - val_acc: 0.9589412962334578
Repeatition 1 Epoch 187 / 500 
 - time: 1.2449934482574463 - sq_loss: 9.458338354306761e-06 - tot_loss: 0.11708549708685467 - acc: 0.9790533188248096 - val_acc: 0.9586019681031558
Repeatition 1 Epoch 188 / 500 
 - time: 1.2773523330688477 - sq_loss: 9.404431693837978e-06 - tot_loss: 0.11025949060682905 - acc: 0.9789173014145811 - val_acc: 0.9589412962334578
Repeatition 1 Epoch 189 / 500 
 - time: 1.2665555477142334 - sq_loss: 9.347408195026219e-06 - tot_loss: 0.10736447685264494 - acc: 0.9789173014145811 - val_acc: 0.9586019681031558
Repeatition 1 Epoch 190 / 500 
 - time: 1.2415809631347656 - sq_loss: 9.298000804847106e-06 - tot_loss: 0.10780876385531712 - acc: 0.9791893362350381 - val_acc: 0.9589412962334578
Repeatition 1 Epoch 191 / 500 
 - time: 1.2411887645721436 - sq_loss: 9.248606147593819e-06 - tot_lo

...

Repeatition 1 Epoch 278 / 500 
 - time: 1.2205634117126465 - sq_loss: 6.7819073592545465e-06 - tot_loss: 0.10183749048830393 - acc: 0.9854461371055495 - val_acc: 0.9613165931455717
Repeatition 1 Epoch 279 / 500 
 - time: 1.2388017177581787 - sq_loss: 6.763453257008223e-06 - tot_loss: 0.09782757872394399 - acc: 0.9854461371055495 - val_acc: 0.9613165931455717
Repeatition 1 Epoch 280 / 500 
 - time: 1.2413408756256104 - sq_loss: 6.746367034793366e-06 - tot_loss: 0.10236440442871952 - acc: 0.9854461371055495 - val_acc: 0.9613165931455717
Repeatition 1 Epoch 281 / 500 
 - time: 1.2415051460266113 - sq_loss: 6.729306733177509e-06 - tot_loss: 0.09971914876850363 - acc: 0.9854461371055495 - val_acc: 0.9613165931455717
Repeatition 1 Epoch 282 / 500 
 - time: 1.2300364971160889 - sq_loss: 6.7114456214767415e-06 - tot_loss: 0.09989926837841523 - acc: 0.9854461371055495 - val_acc: 0.9616559212758737
Repeatition 1 Epoch 283 / 500 
 - time: 1.2245397567749023 - sq_loss: 6.692128863505786e-06 - tot_

Repeatition 1 Epoch 324 / 500 
 - time: 1.2137291431427002 - sq_loss: 6.058982307877159e-06 - tot_loss: 0.09328595190620703 - acc: 0.9862622415669206 - val_acc: 0.9626739056667798
Repeatition 1 Epoch 325 / 500 
 - time: 1.2297694683074951 - sq_loss: 6.0437550928327255e-06 - tot_loss: 0.10205912958618768 - acc: 0.9863982589771491 - val_acc: 0.9626739056667798
Repeatition 1 Epoch 326 / 500 
 - time: 1.2344403266906738 - sq_loss: 6.028583811712451e-06 - tot_loss: 0.09253274060323236 - acc: 0.9863982589771491 - val_acc: 0.9626739056667798
Repeatition 1 Epoch 327 / 500 
 - time: 1.2305340766906738 - sq_loss: 6.014894097461365e-06 - tot_loss: 0.09737559111259131 - acc: 0.9866702937976061 - val_acc: 0.9626739056667798
Repeatition 1 Epoch 328 / 500 
 - time: 1.2256097793579102 - sq_loss: 5.998829237796599e-06 - tot_loss: 0.09420877079149292 - acc: 0.9865342763873776 - val_acc: 0.9633525619273838
Repeatition 1 Epoch 329 / 500 
 - time: 1.2282326221466064 - sq_loss: 5.986018095427426e-06 - tot_l

Repeatition 1 Epoch 370 / 500 
 - time: 1.2322168350219727 - sq_loss: 5.499925464391708e-06 - tot_loss: 0.09513189879212192 - acc: 0.9883025027203483 - val_acc: 0.9630132337970818
Repeatition 1 Epoch 371 / 500 
 - time: 1.2291920185089111 - sq_loss: 5.490118837769842e-06 - tot_loss: 0.0976454040745729 - acc: 0.9880304678998912 - val_acc: 0.9630132337970818
Repeatition 1 Epoch 372 / 500 
 - time: 1.2293376922607422 - sq_loss: 5.480395884660538e-06 - tot_loss: 0.09750394940238394 - acc: 0.9883025027203483 - val_acc: 0.9630132337970818
Repeatition 1 Epoch 373 / 500 
 - time: 1.2329034805297852 - sq_loss: 5.469882580655394e-06 - tot_loss: 0.09314776426242588 - acc: 0.9884385201305768 - val_acc: 0.9630132337970818
Repeatition 1 Epoch 374 / 500 
 - time: 1.239159345626831 - sq_loss: 5.460422471514903e-06 - tot_loss: 0.09703861353646559 - acc: 0.9883025027203483 - val_acc: 0.9633525619273838
Repeatition 1 Epoch 375 / 500 
 - time: 1.264087438583374 - sq_loss: 5.450306616694434e-06 - tot_loss:

Repeatition 1 Epoch 416 / 500 
 - time: 1.2491366863250732 - sq_loss: 5.047989361628424e-06 - tot_loss: 0.0884987821546872 - acc: 0.9895266594124048 - val_acc: 0.9647098744485918
Repeatition 1 Epoch 417 / 500 
 - time: 1.2441489696502686 - sq_loss: 5.038256858824752e-06 - tot_loss: 0.09114577893776854 - acc: 0.9895266594124048 - val_acc: 0.9647098744485918
Repeatition 1 Epoch 418 / 500 
 - time: 1.251774549484253 - sq_loss: 5.02888860864914e-06 - tot_loss: 0.09537475723752209 - acc: 0.9895266594124048 - val_acc: 0.9647098744485918
Repeatition 1 Epoch 419 / 500 
 - time: 1.257833480834961 - sq_loss: 5.019191121391486e-06 - tot_loss: 0.09284385812165397 - acc: 0.9895266594124048 - val_acc: 0.9643705463182898
Repeatition 1 Epoch 420 / 500 
 - time: 1.2553231716156006 - sq_loss: 5.010731911170296e-06 - tot_loss: 0.10345236482678644 - acc: 0.9895266594124048 - val_acc: 0.9647098744485918
Repeatition 1 Epoch 421 / 500 
 - time: 1.268956184387207 - sq_loss: 5.002052603231277e-06 - tot_loss: 0

Repeatition 1 Epoch 462 / 500 
 - time: 1.2450413703918457 - sq_loss: 4.682982307713246e-06 - tot_loss: 0.09470410016142239 - acc: 0.9903427638737758 - val_acc: 0.9657278588394977
Repeatition 1 Epoch 463 / 500 
 - time: 1.2189934253692627 - sq_loss: 4.675995569414226e-06 - tot_loss: 0.10070217247001523 - acc: 0.9903427638737758 - val_acc: 0.9657278588394977
Repeatition 1 Epoch 464 / 500 
 - time: 1.2209296226501465 - sq_loss: 4.669065219786717e-06 - tot_loss: 0.09079502107853266 - acc: 0.9903427638737758 - val_acc: 0.9657278588394977
Repeatition 1 Epoch 465 / 500 
 - time: 1.2266414165496826 - sq_loss: 4.660818376578391e-06 - tot_loss: 0.09240766419779156 - acc: 0.9903427638737758 - val_acc: 0.9657278588394977
Repeatition 1 Epoch 466 / 500 
 - time: 1.2372925281524658 - sq_loss: 4.651935341826174e-06 - tot_loss: 0.09575025693953165 - acc: 0.9903427638737758 - val_acc: 0.9660671869697998
Repeatition 1 Epoch 467 / 500 
 - time: 1.2426824569702148 - sq_loss: 4.642789463105146e-06 - tot_lo